# Tuning curves

Plot tuning curves wrt `s`, `v` and `omega` (and derivatives) for all cells.

## Collect data
Across all bouts in both directions. 

In [1]:
# imports
import sys

from pathlib import Path
import numpy as np
import pandas as pd
import warnings 
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

from fcutils.maths import derivative
sys.path.append("./")
sys.path.append(r"C:\Users\Federico\Documents\GitHub\pysical_locomotion")

from analysis.ephys.utils import get_recording_names, get_data, get_session_bouts, trim_bouts

cache = Path(r"D:\Dropbox (UCL)\Rotation_vte\Locomotion\analysis\ephys\tuning_curves\cache")


[06/14/22 15:40:32] DEBUG    DPL - Saving data and logs to logs                   ]8;id=117166;file://c:\ProgramData\Miniconda3\envs\dev\lib\site-packages\tpd\core.py\core.py]8;;\:]8;id=4581;file://c:\ProgramData\Miniconda3\envs\dev\lib\site-packages\tpd\core.py#118\118]8;;\

                    DEBUG    Saving log file to: logs\log.log                     ]8;id=484130;file://c:\ProgramData\Miniconda3\envs\dev\lib\site-packages\tpd\core.py\core.py]8;;\:]8;id=213411;file://c:\ProgramData\Miniconda3\envs\dev\lib\site-packages\tpd\core.py#119\119]8;;\

Connecting root@127.0.0.1:3306


[06/14/22 15:40:34] DEBUG    Connected to database: LOCOMOTION4                ]8;id=968491;file://C:\Users\Federico\Documents\GitHub\pysical_locomotion\data\dbase\__init__.py\__init__.py]8;;\:]8;id=153835;file://C:\Users\Federico\Documents\GitHub\pysical_locomotion\data\dbase\__init__.py#11\11]8;;\

In [2]:
dt1, dt2 = .25, .50
dt1_frames = int(dt1 * 60)
dt2_frames = int(dt2 * 60)


def get_rec_data(recording:str, bins:dict) -> pd.DataFrame:
    """
        Get all data for a recording.

        Returns a dataframe with speed, angvel... and firing rate of
        each unit at every frame from all locomotion bout.
    """
    print(f"Getting data for {recording}")
    units, left_fl, right_fl, left_hl, right_hl, body = get_data(recording)
    units = units.loc[units.brain_region.isin(["MOs", "MOs1", "MOs2/3", "MOs5", "MOs6a", "MOs6b"])]
    L = len(body.speed)

    out_bouts = get_session_bouts(recording, complete=None)
    in_bouts = get_session_bouts(recording, direction="inbound", complete=None)


    rec_data = {
        **{k:[] for k in bins.keys()},
        **{unit.unit_id:[] for i, unit in units.iterrows()}
    }
    for i, unit in units.iterrows():
        for rep in range(100):
            rec_data[f"{unit.unit_id}_shuffle_{rep}"] = []

    # get data into a single big dataframe
    for bouts in (out_bouts, in_bouts):
        for i, bout in bouts.iterrows():
            duration = bout.end_frame - bout.start_frame
            if bout.end_frame + dt2_frames >= L:
                continue

            speed = body.speed[bout.start_frame : bout.end_frame]
            angvel = body.thetadot[bout.start_frame : bout.end_frame]
            s = bout.s[bout.start_frame - bout.start_frame : bout.end_frame - bout.start_frame]
            sdot = derivative(s) * 60

            # get change in angular velocity and speed over two time intervals
            speed_shift_one = body.speed[bout.start_frame + dt1_frames : bout.end_frame + dt1_frames]
            angvel_shift_one = body.thetadot[bout.start_frame + dt1_frames : bout.end_frame + dt1_frames]
            speed_shift_two = body.speed[bout.start_frame + dt2_frames : bout.end_frame + dt2_frames]
            angvel_shift_two = body.thetadot[bout.start_frame + dt2_frames : bout.end_frame + dt2_frames]

            # store behavioral variables
            rec_data['s'].extend(s)
            rec_data['sdot'].extend(sdot)
            rec_data['speed'].extend(speed)
            rec_data['angular_velocity'].extend(angvel)
            rec_data["dspeed_250ms"].extend(speed_shift_one - speed)
            rec_data["dspeed_500ms"].extend(speed_shift_two - speed)
            rec_data["dangvel_250ms"].extend(angvel_shift_one - angvel)
            rec_data["dangvel_500ms"].extend(angvel_shift_two - angvel)

            # get firing rate of unit and of time shifted shuffled data
            for n, unit in units.iterrows():
                frate = unit.firing_rate[bout.start_frame : bout.end_frame]
                rec_data[unit.unit_id].extend(frate)


                for rep in range(100):

                    shift = int((np.random.random() * 200 - 100) * 60)  # random shift in [-100, 100] seconds
                    if (bout.end_frame + shift) > L - 20 * 60 and shift > 0:
                        shift = - shift
                    if (bout.start_frame + shift) < 20 * 60 and shift < 0:
                        shift = - shift

                    start = bout.start_frame + shift
                    end = start + duration

                    assert end - start == duration, f"{shift} {end-start} {duration}"

                    rec_data[f"{unit.unit_id}_shuffle_{rep}"].extend(unit.firing_rate[start:end])

    # print({k:len(v) for k, v in rec_data.items()})
    print(set((len(v) for v in rec_data.values())))
    out =  pd.DataFrame(rec_data)
    out.to_hdf(cache / f"{recording}.h5", key="hdf")

In [3]:
"""
Store a dictionary with a dataframe for each of the variables. 
In these datafranes each row is a unit and each column is the firing rate
at each column is a bin with associated mean/std of firing rate.

"""

bins = dict(
    s = np.linspace(0, 260, 21),
    sdot = np.linspace(-80, 80, 21),
    speed = np.linspace(10, 80, 21),
    dspeed_250ms = np.linspace(-60, 60, 41),
    dspeed_500ms = np.linspace(-60, 60, 41),
    angular_velocity = np.linspace(-400, 400, 21),
    dangvel_250ms = np.linspace(-600, 600, 21),
    dangvel_500ms = np.linspace(-600, 600, 21),
)



# for REC in get_recording_names():
#     get_rec_data(REC, bins)

### Bin data

First bin based on the values of each selected variable, then for each bin group the data into chunks of N frames and take the average and variance of the firing rate in each group.

In [ ]:
def sample_and_bin(rec_data, var, bins):
    print(f"    binning {var}")
    data = rec_data.copy().reset_index()

    # bin based on the variable of interest
    _bins = pd.cut(data[var], bins[var])
    data['bin'] = [b.mid if isinstance(b, pd.Interval) else np.nan for b in _bins.values]
    groups = data.groupby(_bins)
    
    binned_sampled_mean, binned_sample_var = [], []
    for _, group in groups:
        group.reset_index(inplace=True)
        # split into groupps of equal length
        splits = group.groupby(group.index // 300)
        for _, split in splits:
            if len(group) < 300:
                continue
            binned_sampled_mean.append(split.mean())
            binned_sample_var.append(split.var())
        
    # return pd.concat(binned_sampled_mean, ignore_index=False), pd.concat(binned_sample_var, ignore_index=False)
    return pd.concat(binned_sampled_mean, axis=1).T, pd.concat(binned_sample_var, axis=1).T



In [ ]:
for rec in get_recording_names():
    print(rec)
    rec_data = pd.read_hdf(cache / (rec + ".h5"), key="hdf")
    

    # bin the recording data for each variable
    for k in bins.keys():
        mu, sigma = sample_and_bin(rec_data, k, bins)
        mu.to_hdf(cache / (f"{rec}_{k}_mu.h5"), key="hdf")
        sigma.to_hdf(cache / (f"{rec}_{k}_sigma.h5"), key="hdf")
   
